# 🤖 Agents e Tools: Dando "Superpoderes" para seus LLMs!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-11_img_01.png)

**Pedro Nunes Guth - Módulo 11 de 17: LangChain v1.0**

---

Tá, vamos falar sério aqui! Até agora nossos LLMs eram meio que aquele amigo inteligente, mas que só sabe conversar. E se eu te disser que podemos dar "ferramentas" para ele usar?

Imagina o seguinte: você pergunta "Quanto é 2534 x 8971?" e em vez do LLM "chutar" o resultado, ele literalmente usa uma calculadora! Ou você pede "Me diga o clima em São Paulo" e ele consulta uma API de verdade!

**Isso é o que vamos aprender hoje: Agents e Tools!**

Bora! 🚀

## 🎯 O que são Agents?

Cara, pensa no Agent como aquele assistente super eficiente que não só conversa contigo, mas também **toma decisões** sobre quais ferramentas usar!

### Analogia do Garçom Inteligente 🍽️

Imagina um garçom em um restaurante que:
1. **Escuta** seu pedido
2. **Decide** se precisa consultar o cardápio, verificar estoque ou falar com o chef
3. **Usa as ferramentas** necessárias (tablet, telefone, vai até a cozinha)
4. **Te dá** a resposta completa

É isso que um Agent faz! Ele decide qual ferramenta usar baseado no que você perguntou.

### Diferença Crucial:

- **LLM simples**: "Acho que 2+2 = 4" (pode errar cálculos complexos)
- **Agent com calculadora**: Usa calculadora de verdade → "2+2 = 4" (sempre certo)

**Dica!** Agents são perfeitos quando você precisa de informações atuais ou cálculos precisos!

## ⚒️ O que são Tools?

Tools (ferramentas) são as "habilidades especiais" que você dá para o Agent usar!

### Tipos de Tools Populares:

1. **🧮 Calculadora** - Para matemática precisa
2. **🌐 Web Search** - Buscar info na internet
3. **📊 API Calls** - Consultar serviços externos
4. **📝 File Operations** - Ler/escrever arquivos
5. **🗃️ Database** - Consultar bancos de dados

### Analogia da Caixa de Ferramentas 🧰

É como dar uma caixa de ferramentas para um marceneiro:
- Tem um parafuso? Usa a chave de fenda
- Precisa cortar? Usa a serra
- Quer medir? Usa a régua

O Agent **escolhe automaticamente** qual ferramenta usar!

**Dica!** Cada tool tem uma "descrição" que ensina o Agent quando usá-la!

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# !pip install langchain langchain-google-genai langchain-community python-dotenv -q

# ===== VERSÃO 1.0 =====
# Bora começar! Instalando as dependências
# NOTA: Em LangChain 1.0, Agents foram movidos para LangGraph!
!pip install langchain>=1.0.0 langchain-core langchain-community langchain-google-genai langgraph python-dotenv -q

print("📦 Pacotes instalados com sucesso!")
print("🚀 Pronto para criar nossos primeiros Agents!")

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain.tools import Tool
# from langchain_core.prompts import ChatPromptTemplate

# ===== VERSÃO 1.0 =====
# Configuração inicial - Lembra dos módulos anteriores?
# NOTA: Em LangChain 1.0, usamos create_agent de langchain.agents!
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent  # Nova API em v1.0!
from langchain_core.tools import tool, Tool  # Decorator e classe para criar tools
from langchain_core.prompts import ChatPromptTemplate

# Carrega variáveis de ambiente
load_dotenv()

# Configurar a API key do Google (você já aprendeu isso no módulo 2!)
# Coloque sua chave aqui ou use .env
# os.environ["GOOGLE_API_KEY"] = "sua_api_key_aqui"  # Substitua pela sua chave

# Criando nosso modelo (igual ao módulo 2)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0  # Zero para ser mais preciso
)

print("✅ LLM configurado!")
print("🎯 Modelo: gemini-2.0-flash (nosso queridinho do curso)")

## 🧮 Criando Nossa Primeira Tool: Calculadora

Vamos criar uma calculadora simples para mostrar como funciona!

### Como funciona uma Tool:

1. **Nome**: Como a ferramenta se chama
2. **Descrição**: Quando o Agent deve usá-la
3. **Função**: O que ela realmente faz

É tipo ensinar uma criança: "Esta ferramenta se chama 'calculadora', use ela quando precisar fazer contas, e ela funciona assim..."

In [ ]:
# Criando uma função simples de calculadora
def calculadora(expressao):
    """
    Função que calcula expressões matemáticas simples
    Exemplo: "2 + 2" retorna "4"
    """
    try:
        # eval() é meio perigoso em produção, mas para exemplo tá ok
        resultado = eval(expressao)
        return f"O resultado de {expressao} é {resultado}"
    except Exception as e:
        return f"Erro no cálculo: {str(e)}"

# Testando nossa função
print("🧪 Testando a calculadora:")
print(calculadora("2 + 2"))
print(calculadora("10 * 5"))
print(calculadora("100 / 4"))

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# from langchain.tools import Tool

# ===== VERSÃO 1.0 =====
# Agora vamos transformar nossa função em uma Tool do LangChain
# Em v1.0, Tool está em langchain_core.tools
calculator_tool = Tool(
    name="Calculadora",
    description="Use esta ferramenta quando precisar fazer cálculos matemáticos. Entrada deve ser uma expressão matemática como '2+2' ou '10*5'.",
    func=calculadora
)

# Vamos criar mais uma tool útil: informações sobre o Brasil
def info_brasil(pergunta):
    """
    Banco de dados simples sobre o Brasil
    """
    info_db = {
        "capital": "A capital do Brasil é Brasília",
        "população": "O Brasil tem aproximadamente 215 milhões de habitantes",
        "moeda": "A moeda do Brasil é o Real (BRL)",
        "idioma": "O idioma oficial do Brasil é o Português",
        "area": "O Brasil tem 8.515.767 km² de área"
    }

    # Busca a informação mais próxima
    pergunta_lower = pergunta.lower()
    for chave, resposta in info_db.items():
        if chave in pergunta_lower:
            return resposta

    return "Informação não encontrada. Posso ajudar com: capital, população, moeda, idioma, area"

# Criando a segunda tool
brasil_tool = Tool(
    name="Info_Brasil",
    description="Use esta ferramenta para obter informações básicas sobre o Brasil: capital, população, moeda, idioma, área.",
    func=info_brasil
)

print("🛠️ Tools criadas com sucesso!")
print("📋 Ferramentas disponíveis:")
print(f"1. {calculator_tool.name}: {calculator_tool.description}")
print(f"2. {brasil_tool.name}: {brasil_tool.description}")

## 🤖 Criando Nosso Primeiro Agent

Agora vem a parte mais legal! Vamos juntar tudo:
- **LLM** (já configurado)
- **Tools** (acabamos de criar)
- **Prompt** (você já sabe do módulo 4!)

### O Agent vai:
1. **Analisar** sua pergunta
2. **Decidir** qual tool usar (ou se não precisa de nenhuma)
3. **Executar** a tool se necessário
4. **Responder** com o resultado

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-11_img_02.png)

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# agent = create_tool_calling_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     verbose=True,
#     handle_parsing_errors=True
# )

# ===== VERSÃO 1.0 =====
# Primeiro, vamos definir nossas tools e o system prompt
tools = [calculator_tool, brasil_tool]

system_prompt = """Você é um assistente inteligente com acesso a ferramentas especiais!

🧮 CALCULADORA: Use para qualquer cálculo matemático
🇧🇷 INFO_BRASIL: Use para informações sobre o Brasil

Sempre use as ferramentas quando necessário para dar respostas precisas."""

# Criando o agent usando create_agent (a mágica acontece aqui!)
# Em LangChain 1.0, create_agent é a forma recomendada!
agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt  # System prompt customizado
)

print("🎉 AGENT CRIADO COM SUCESSO!")
print("🔧 Ferramentas disponíveis:", [tool.name for tool in tools])
print("✅ Pronto para usar!")

In [ ]:
# Célula anterior já criou o agent - esta é apenas uma verificação
print("✅ Agent já criado na célula anterior!")
print(f"🔧 Tools disponíveis: {[t.name for t in tools]}")

## 🧪 Testando Nosso Agent

Hora do show! Vamos testar nosso agent com diferentes tipos de pergunta para ver ele escolhendo as ferramentas certas!

### Vamos testar:
1. **Cálculo matemático** → Deve usar a Calculadora
2. **Pergunta sobre Brasil** → Deve usar Info_Brasil  
3. **Pergunta geral** → Não deve usar nenhuma tool

**Dica!** Repare como o `verbose=True` vai mostrar o "raciocínio" do agent!

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# resposta1 = agent_executor.invoke({"input": pergunta1})
# print(f"🤖 Resposta: {resposta1['output']}")

# ===== VERSÃO 1.0 =====
# Teste 1: Cálculo matemático
print("🧮 TESTE 1: Cálculo Matemático")
print("="*50)

pergunta1 = "Quanto é 1234 * 5678?"

# Em LangGraph, usamos invoke com "messages"
resposta1 = agent_executor.invoke({"messages": [("user", pergunta1)]})

print(f"\n💬 Pergunta: {pergunta1}")
# A resposta em LangGraph vem no formato de messages
print(f"🤖 Resposta: {resposta1['messages'][-1].content}")
print("\n" + "="*50)

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# resposta2 = agent_executor.invoke({"input": pergunta2})
# print(f"🤖 Resposta: {resposta2['output']}")

# ===== VERSÃO 1.0 =====
# Teste 2: Informação sobre o Brasil
print("🇧🇷 TESTE 2: Informações sobre o Brasil")
print("="*50)

pergunta2 = "Qual é a capital do Brasil?"
resposta2 = agent_executor.invoke({"messages": [("user", pergunta2)]})

print(f"\n💬 Pergunta: {pergunta2}")
print(f"🤖 Resposta: {resposta2['messages'][-1].content}")
print("\n" + "="*50)

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# resposta3 = agent_executor.invoke({"input": pergunta3})
# print(f"🤖 Resposta: {resposta3['output']}")

# ===== VERSÃO 1.0 =====
# Teste 3: Pergunta geral (não deve usar nenhuma tool)
print("💭 TESTE 3: Pergunta Geral")
print("="*50)

pergunta3 = "O que você acha do futebol brasileiro?"
resposta3 = agent_executor.invoke({"messages": [("user", pergunta3)]})

print(f"\n💬 Pergunta: {pergunta3}")
print(f"🤖 Resposta: {resposta3['messages'][-1].content}")
print("\n" + "="*50)

# Simulando uma base de "web search" com informações atualizadas
def web_search_simulada(query):
    """
    Simula uma busca na web com informações "atuais"
    Em produção, isso seria uma API real do Google, Bing, etc.
    """

    # Base de dados simulada com informações "recentes"
    resultados_web = {
        "clima": "Hoje em São Paulo: 28°C, parcialmente nublado. Fonte: Weather API",
        "dólar": "Cotação atual do dólar: R$ 5,85. Última atualização: hoje 15h30",
        "copa": "Próxima Copa do Mundo será em 2026 nos EUA, Canadá e México",
        "bitcoin": "Bitcoin hoje: $43,250 USD (+2.3% nas últimas 24h)",
        "eleições": "Próximas eleições municipais no Brasil: 2024",
        "feriado": "Próximo feriado nacional: Carnaval (fevereiro 2024)"
    }

    query_lower = query.lower()

    # Busca por palavras-chave
    for palavra_chave, resultado in resultados_web.items():
        if palavra_chave in query_lower:
            return f"🔍 Resultado da busca: {resultado}"

    # Se não encontrar nada específico
    return "🔍 Busca realizada, mas não encontrei informações específicas sobre isso. Posso ajudar com: clima, dólar, copa, bitcoin, eleições, feriado"

# Criando a tool de web search
web_search_tool = Tool(
    name="Web_Search",
    description="Use esta ferramenta para buscar informações atuais na internet como clima, cotações, notícias recentes, etc. Útil quando o usuário pergunta sobre dados em tempo real.",
    func=web_search_simulada
)

# Testando a nova tool
print("🌐 Testando a Web Search:")
print(web_search_simulada("qual a cotação do dólar hoje?"))
print(web_search_simulada("como está o clima em São Paulo?"))

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# prompt_avancado = ChatPromptTemplate.from_messages([...])
# agent_avancado = create_tool_calling_agent(llm, tools_avancadas, prompt_avancado)
# agent_executor_avancado = AgentExecutor(...)

# ===== VERSÃO 1.0 =====
# Criando um Agent mais poderoso com 3 tools!
tools_avancadas = [calculator_tool, brasil_tool, web_search_tool]

# System prompt mais elaborado
system_prompt_avancado = """Você é um assistente super inteligente com acesso a ferramentas especiais!

🧮 CALCULADORA: Para qualquer cálculo matemático
🇧🇷 INFO_BRASIL: Para informações básicas sobre o Brasil
🌐 WEB_SEARCH: Para informações atuais (clima, cotações, notícias)

INSTRUÇÕES:
1. Analise a pergunta do usuário
2. Decida qual ferramenta usar (ou nenhuma)
3. Execute a ferramenta se necessário
4. Dê uma resposta completa e amigável

Seja sempre educado e explique o que você fez!"""

# Criando o agent avançado usando create_agent (v1.0)
agent_executor_avancado = create_agent(
    model=llm,
    tools=tools_avancadas,
    system_prompt=system_prompt_avancado
)

print("🚀 AGENT AVANÇADO CRIADO!")
print(f"🔧 {len(tools_avancadas)} ferramentas disponíveis:")
for i, tool in enumerate(tools_avancadas, 1):
    print(f"  {i}. {tool.name}")

# Célula anterior já criou o agent avançado - esta é apenas uma verificação
print("✅ Agent avançado já criado!")
print(f"🔧 Tools: {[t.name for t in tools_avancadas]}")

In [ ]:
# Célula duplicada - web_search_tool já foi definido acima
print("✅ web_search_tool já definido!")

In [ ]:
# Célula duplicada - agent_executor_avancado já foi criado acima
print("✅ agent_executor_avancado já criado!")

# ===== VERSÃO 0.3 (comentada) =====
# agent_final = create_tool_calling_agent(llm, todas_tools, prompt_avancado)
# agent_executor_final = AgentExecutor(...)
# resultado = agent_executor_final.invoke({"input": "..."})

# ===== VERSÃO 1.0 =====
# EXERCÍCIO 2: Criando a tool e adicionando ao agent

# Crie a tool de temperatura
temperatura_tool = Tool(
    name="Conversor_Temperatura",
    description="Use para converter temperaturas entre Celsius e Fahrenheit. Aceita formatos como '25C', '77F', 'celsius', 'fahrenheit'.",
    func=conversor_temperatura
)

# Agent com 4 tools agora!
todas_tools = [calculator_tool, brasil_tool, web_search_tool, temperatura_tool]

# Criando agent final usando create_agent (v1.0)
agent_executor_final = create_agent(
    model=llm,
    tools=todas_tools,
    system_prompt=system_prompt_avancado
)

print("🎉 AGENT FINAL CRIADO!")
print(f"🔧 Agora temos {len(todas_tools)} ferramentas:")
for i, tool in enumerate(todas_tools, 1):
    print(f"  {i}. {tool.name}")

# Teste a nova funcionalidade
print("\n🧪 Testando conversão de temperatura:")
resultado = agent_executor_final.invoke({"messages": [("user", "Converte 30 graus celsius para fahrenheit")]})
print(f"Resultado: {resultado['messages'][-1].content}")

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# resposta = agent_executor_avancado.invoke({"input": pergunta})
# print(f"✅ Resposta: {resposta['output']}")

# ===== VERSÃO 1.0 =====
# Teste com múltiplas perguntas para ver a escolha de tools
perguntas_teste = [
    "Quanto é 987 + 654 - 321?",
    "Qual é a cotação do dólar hoje?",
    "Quantos habitantes tem o Brasil?",
    "Como está o clima em São Paulo?",
    "Qual sua cor favorita?"  # Esta não deve usar nenhuma tool
]

print("🧪 BATERIA DE TESTES DO AGENT AVANÇADO")
print("="*60)

for i, pergunta in enumerate(perguntas_teste, 1):
    print(f"\n🔸 TESTE {i}: {pergunta}")
    print("-" * 40)

    try:
        resposta = agent_executor_avancado.invoke({"messages": [("user", pergunta)]})
        print(f"✅ Resposta: {resposta['messages'][-1].content}")
    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

# Célula duplicada - agent_executor_final já criado na célula 20
print("✅ agent_executor_final já criado!")

In [ ]:
# Célula duplicada - agent_executor_final já criado na célula 20
print("✅ agent_executor_final já criado!")

# ===== VERSÃO 0.3 (comentada) =====
# resposta_agent = agent_executor_final.invoke({"input": pergunta_complexa})
# print(f"Resposta: {resposta_agent['output']}")

# ===== VERSÃO 1.0 =====
# Comparação: LLM simples vs Agent com tools

pergunta_complexa = "Quanto é (1234 * 5678) + (9876 / 4) - 321?"

print("🥊 LLM SIMPLES vs AGENT COM TOOLS")
print("="*60)

# 1. Teste com LLM simples (sem tools)
print("\n🤖 LLM SIMPLES (sem ferramentas):")
print("-" * 40)
resposta_simples = llm.invoke(pergunta_complexa)
print(f"Resposta: {resposta_simples.content}")

# 2. Teste com Agent (com calculadora)
print("\n🦾 AGENT COM TOOLS:")
print("-" * 40)
resposta_agent = agent_executor_final.invoke({"messages": [("user", pergunta_complexa)]})
print(f"Resposta: {resposta_agent['messages'][-1].content}")

# Verificação manual do cálculo
resultado_correto = (1234 * 5678) + (9876 / 4) - 321
print(f"\n✅ Resultado correto: {resultado_correto}")
print("\n📊 ANÁLISE:")

print(" LLM simples: Pode chutar ou errar cálculos complexos")
print(" Agent com tools: Usa calculadora real = sempre correto")

In [ ]:
# EXERCÍCIO 1: Sua vez! Crie uma tool de conversão de temperatura

def conversor_temperatura(entrada):
    """
    Converte temperaturas entre Celsius e Fahrenheit
    Exemplos de entrada: "25C", "77F", "0 celsius", "32 fahrenheit"

    Complete esta função!
    """
    # SEU CÓDIGO AQUI!
    # Dicas:
    # - C para F: (C * 9/5) + 32
    # - F para C: (F - 32) * 5/9
    # - Use .upper(), .replace(), split() para processar a entrada

    try:
        entrada_clean = entrada.upper().replace(" ", "")

        if "C" in entrada_clean:
            # Extrair número e converter para Fahrenheit
            numero = float(entrada_clean.replace("C", "").replace("CELSIUS", ""))
            fahrenheit = (numero * 9/5) + 32
            return f"{numero}°C = {fahrenheit:.1f}°F"

        elif "F" in entrada_clean:
            # Extrair número e converter para Celsius
            numero = float(entrada_clean.replace("F", "").replace("FAHRENHEIT", ""))
            celsius = (numero - 32) * 5/9
            return f"{numero}°F = {celsius:.1f}°C"

        else:
            return "Formato inválido. Use: '25C', '77F', '0 celsius' ou '32 fahrenheit'"

    except Exception as e:
        return f"Erro na conversão: {str(e)}"

# Teste sua função
print("🧪 Testando o conversor:")
print(conversor_temperatura("25C"))
print(conversor_temperatura("77F"))
print(conversor_temperatura("0 celsius"))

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# agent_final = create_tool_calling_agent(llm, todas_tools, prompt_avancado)
# agent_executor_final = AgentExecutor(...)
# resultado = agent_executor_final.invoke({"input": "..."})

# ===== VERSÃO 1.0 =====
# EXERCÍCIO 2: Criando a tool e adicionando ao agent

# Crie a tool de temperatura
temperatura_tool = Tool(
    name="Conversor_Temperatura",
    description="Use para converter temperaturas entre Celsius e Fahrenheit. Aceita formatos como '25C', '77F', 'celsius', 'fahrenheit'.",
    func=conversor_temperatura
)

# Agent com 4 tools agora!
todas_tools = [calculator_tool, brasil_tool, web_search_tool, temperatura_tool]

# Criando agent final usando create_agent (v1.0)
agent_executor_final = create_agent(
    model=llm,
    tools=todas_tools,
    system_prompt=system_prompt_avancado
)

print("🎉 AGENT FINAL CRIADO!")
print(f"🔧 Agora temos {len(todas_tools)} ferramentas:")
for i, tool in enumerate(todas_tools, 1):
    print(f"  {i}. {tool.name}")

# Teste a nova funcionalidade
print("\n🧪 Testando conversão de temperatura:")
resultado = agent_executor_final.invoke({"messages": [("user", "Converte 30 graus celsius para fahrenheit")]})
print(f"Resultado: {resultado['messages'][-1].content}")

## 📊 Comparação: Com vs Sem Agents

Vamos ver a diferença prática entre usar um LLM simples vs um Agent com tools!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-11_img_04.png)

### Teste Prático: Cálculo Complexo

In [ ]:
# ===== VERSÃO 0.3 (comentada) =====
# resposta_agent = agent_executor_final.invoke({"input": pergunta_complexa})
# print(f"Resposta: {resposta_agent['output']}")

# ===== VERSÃO 1.0 =====
# Comparação: LLM simples vs Agent com tools

pergunta_complexa = "Quanto é (1234 * 5678) + (9876 / 4) - 321?"

print("🥊 LLM SIMPLES vs AGENT COM TOOLS")
print("="*60)

# 1. Teste com LLM simples (sem tools)
print("\n🤖 LLM SIMPLES (sem ferramentas):")
print("-" * 40)
resposta_simples = llm.invoke(pergunta_complexa)
print(f"Resposta: {resposta_simples.content}")

# 2. Teste com Agent (com calculadora)
print("\n🦾 AGENT COM TOOLS:")
print("-" * 40)
resposta_agent = agent_executor_final.invoke({"messages": [("user", pergunta_complexa)]})
print(f"Resposta: {resposta_agent['messages'][-1].content}")

# Verificação manual do cálculo
resultado_correto = (1234 * 5678) + (9876 / 4) - 321
print(f"\n✅ Resultado correto: {resultado_correto}")
print("\n📊 ANÁLISE:")

print(" LLM simples: Pode chutar ou errar cálculos complexos")
print(" Agent com tools: Usa calculadora real = sempre correto")

## 🚀 Preparando para os Próximos Módulos

Liiiindo! Você acabou de dominar Agents e Tools! Mas isso é só o começo...

### 🔮 O que vem por aí:

**Módulo 12-13: Projetos Finais**
- Vamos usar Agents em projetos reais!
- Combinar com RAG (módulo 10) + Memory (módulo 7)
- Chatbots inteligentes que fazem coisas de verdade

**Módulo 14: Deploy e Produção**
- Colocar seus Agents no ar com Streamlit
- Interface bonita para usuários finais

**Módulo 16: LangGraph**
- Agents ainda mais sofisticados
- Fluxos complexos de decisão
- Multi-agents trabalhando juntos!

### 💡 Conexões com módulos anteriores:
- **Prompt Templates** (Módulo 4): Essenciais para configurar agents
- **Memory** (Módulo 7): Agents que lembram de conversas passadas
- **RAG** (Módulo 10): Agents que consultam documentos específicos

**Dica!** Agents são o "cérebro" que conecta tudo que aprendemos até agora!

In [ ]:
# Visualização final: Evolução dos nossos LLMs no curso
import matplotlib.pyplot as plt
import numpy as np

modulos = ['Módulo 2\nChatModel', 'Módulo 4\nPrompt Template', 'Módulo 7\nMemory', 'Módulo 10\nRAG', 'Módulo 11\nAgents']
capacidades = [1, 2, 4, 7, 9]  # Nível de capacidade (1-10)
complexidade = [1, 2, 3, 6, 8]  # Nível de complexidade

fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(modulos))
width = 0.35

bars1 = ax.bar(x - width/2, capacidades, width, label='Capacidades do LLM',
               color='#4CAF50', alpha=0.8)
bars2 = ax.bar(x + width/2, complexidade, width, label='Complexidade de Uso',
               color='#FF9800', alpha=0.8)

ax.set_xlabel('Módulos do Curso', fontsize=12)
ax.set_ylabel('Nível (1-10)', fontsize=12)
ax.set_title('🚀 Evolução das Capacidades dos LLMs no Curso', fontsize=16, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(modulos, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📈 EVOLUÇÃO DO CURSO:")
print("✅ Começamos com LLMs básicos")
print("✅ Adicionamos templates e memory")
print("✅ Incorporamos knowledge com RAG")
print("✅ AGORA: Agents que usam ferramentas!")
print("🚀 PRÓXIMO: Projetos reais combinando tudo!")

## 📚 Resumo do Módulo: O que Aprendemos?

Cara, que jornada! Vamos recapitular o que dominamos sobre **Agents e Tools**:

### 🎯 Conceitos Fundamentais:
- **Agent**: O "cérebro" que decide quais ferramentas usar
- **Tools**: As "habilidades especiais" que damos para o LLM
- **Reasoning**: Como o agent "pensa" e toma decisões

### 🛠️ Na Prática Criamos:
1. **Calculadora Tool** - Matemática precisa
2. **Info Brasil Tool** - Base de dados local
3. **Web Search Tool** - Informações "atuais"
4. **Conversor Temperatura** - Funcionalidade personalizada

### 🧠 Processo de Decisão do Agent:
```
Pergunta → Análise → Escolha da Tool → Execução → Resposta
```

### 💡 Principais Aprendizados:
- **Descrições claras** das tools são cruciais
- Agents **combinam** inteligência com **precisão**
- **verbose=True** ajuda a entender o processo
- Tools podem ser **simples ou complexas**

### 🔗 Conexões com o Curso:
- Usamos **ChatModel** (Módulo 2)
- Aplicamos **Prompt Templates** (Módulo 4)
- Preparamos terreno para **projetos finais**

**Dica Final!** Agents são o futuro dos LLMs - eles não só "sabem", mas também "fazem"!

---

**🎉 Parabéns! Você agora sabe criar LLMs que usam ferramentas de verdade!**

**Próxima parada: Módulo 12 - Projeto Final 1** 🚀

Bora!